## Motor Vehicle Collisions

The following notebook downloads the [NYPD Motor Vehicle Collisions dataset](https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95) (part of the NYC Open Data initiative) and cleans it for the purposes of analysis.

In [1]:
import pandas as pd
from pandas import DataFrame
import geocoder
import requests
import arrow
import math
import ratelim
from datetime import datetime
from tqdm import tqdm

In [2]:
# Not currently working: https://github.com/JosPolfliet/pandas-profiling/issues/5
# import pandas_profiling

In [3]:
def download_file(url, filename):
    """
    Helper method handling downloading large files from `url` to `filename`. Returns a pointer to `filename`.
    """
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return filename

In [4]:
dat = download_file("https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD",
                    "NYPD Motor Vehicle Collisions.csv")

In [5]:
# Column 23 is UNIQUE KEY.
df = pd.read_csv(dat, index_col=23)

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
df.head(10)

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
UNIQUE KEY,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3405169,03/14/2016,3:27,QUEENS,11372.0,40.747734,-73.882999,"(40.7477341, -73.8829986)",ROOSEVELT AVENUE,83 STREET,NaN,1,0,1,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,OTHER,NaN,NaN,NaN,NaN
3405070,03/14/2016,2:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Other Vehicular,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,OTHER,NaN,NaN,NaN
3405059,03/14/2016,0:45,MANHATTAN,10035.0,40.808279,-73.938793,"(40.8082795, -73.9387929)",EAST 129 STREET,MADISON AVENUE,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3405134,03/14/2016,2:00,NaN,NaN,NaN,NaN,NaN,PULASKI BRIDGE,EAGLE STREET,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,OTHER,NaN,NaN,NaN
3404661,03/13/2016,1:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
3404800,03/13/2016,1:21,BROOKLYN,11215.0,40.677834,-73.988254,"(40.6778345, -73.9882541)",NEVINS STREET,CARROLL STREET,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3405060,03/13/2016,22:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3405144,03/13/2016,22:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PARKING LOT OF 79-15 MAIN STREET,0,0,0,0,0,0,0,0,Other Vehicular,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3404950,03/13/2016,22:18,QUEENS,11434.0,40.672385,-73.785739,"(40.6723848, -73.7857391)",BAISLEY BOULEVARD,ROCKAWAY BOULEVARD,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


In [8]:
len(df)

769054

In [9]:
df.count(axis=0)

DATE                             769054
TIME                             769054
BOROUGH                          584753
ZIP CODE                         584686
LATITUDE                         647867
LONGITUDE                        647867
LOCATION                         647867
ON STREET NAME                   655435
CROSS STREET NAME                655435
OFF STREET NAME                   23792
NUMBER OF PERSONS INJURED        769054
NUMBER OF PERSONS KILLED         769054
NUMBER OF PEDESTRIANS INJURED    769054
NUMBER OF PEDESTRIANS KILLED     769054
NUMBER OF CYCLIST INJURED        769054
NUMBER OF CYCLIST KILLED         769054
NUMBER OF MOTORIST INJURED       769054
NUMBER OF MOTORIST KILLED        769054
CONTRIBUTING FACTOR VEHICLE 1    767363
CONTRIBUTING FACTOR VEHICLE 2    676675
CONTRIBUTING FACTOR VEHICLE 3     49848
CONTRIBUTING FACTOR VEHICLE 4     10256
CONTRIBUTING FACTOR VEHICLE 5      2569
VEHICLE TYPE CODE 1              768105
VEHICLE TYPE CODE 2              688860


Some observations:
* `UNIQUE KEY` is the `Index`.
* Datetime is split across `Date` and `Time`, Tableau expects them to be combined in its input and messes things up if it is not.
* One of `On Street Name`, `Off Street Name`, `Cross Street Name`, or `Location` is *almost* always present.

  In the exceptional cases there is nothing that can be done with this entry, as it has no identifiable location tag, so we will have to throw the entry out.
  
  Otherwise we can populate all other location-tagged parameters using geocoding or reverse geocoding.
* Contibuting factors, injuries to parties, and datetime information is always complete.

Step one, merge date and time into datetime. Modifying data in-place in terms of `Series` in a `DataFrame` is not fast: takes 21 minutes. It's much faster to write a new Index and then insert it into the table, then drop the original ones.

In [10]:
def create_datetime_axis(daf):
    """
    Pandas ufunc which takes the DataFrame, merging the date and time into a `datetime` axis.
    ex. Series(['03/13/2016', '2:10']) -> Series(['2016:03:13:2:10', NaN])
    """
    datetime_axis = []
    for row_n in tqdm(range(len(daf))):
        srs = daf.iloc[row_n, :]
        datetime_axis.append(datetime.strptime(''.join([srs['DATE'], srs['TIME']]), '%m/%d/%Y%H:%M').strftime('%m/%d/%Y %H:%M'))
    return datetime_axis

Oh well.

In [16]:
filtered_df = df.copy()
# filtered_df = filtered_df.head(10)
filtered_df['DATE'] = create_datetime_axis(filtered_df)
filtered_df.drop('TIME', axis=1, inplace=True)
filtered_df.rename(columns={'DATE':'DATETIME'}, inplace=True)
filtered_df

,DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
UNIQUE KEY,,,,,,,,,,,,,,,,,,,,,,,,,,,
3405169,03-14-2016 03:27,QUEENS,11372.0,40.747734,-73.882999,"(40.7477341, -73.8829986)",ROOSEVELT AVENUE,83 STREET,NaN,1,0,1,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,OTHER,NaN,NaN,NaN,NaN
3405070,03-14-2016 02:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Other Vehicular,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,OTHER,NaN,NaN,NaN
3405059,03-14-2016 00:45,MANHATTAN,10035.0,40.808279,-73.938793,"(40.8082795, -73.9387929)",EAST 129 STREET,MADISON AVENUE,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3405134,03-14-2016 02:00,NaN,NaN,NaN,NaN,NaN,PULASKI BRIDGE,EAGLE STREET,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,OTHER,NaN,NaN,NaN
3404661,03-13-2016 01:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
3404800,03-13-2016 01:21,BROOKLYN,11215.0,40.677834,-73.988254,"(40.6778345, -73.9882541)",NEVINS STREET,CARROLL STREET,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3405060,03-13-2016 22:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3405144,03-13-2016 22:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PARKING LOT OF 79-15 MAIN STREET,0,0,0,0,0,0,0,0,Other Vehicular,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3404950,03-13-2016 22:18,QUEENS,11434.0,40.672385,-73.785739,"(40.6723848, -73.7857391)",BAISLEY BOULEVARD,ROCKAWAY BOULEVARD,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


It would be far too costly to complete the record by geocoding every single missing geographic entry in the dataset, so instead we will geocode only the ones that we absolutely need: reports which resulted in injuries or deaths.

Spot-estimated geocoding API usage for all Injury OR Death reports in the dataset: this eats up to half of the Google Geocoding API's 25000 daily requests rate limit.

This is an estimate of the number of incident records in the dataset which result in injuries or deaths but are filed without coordinate data.

In [17]:
(15/1000)*len(df)

11535.81

Spot-estimated geocoding API usage for all Death (ONLY) reports in the dataset. Incomplete records of deaths occur two orders of magnitude less often than those of injuries.

This is an estimate of the number of incident records in the dataset which result in deaths but are filed without coordinate data.

In [18]:
(2/10000)*len(df)

153.8108

With ~12000 records outstanding, it seems reasonable to assume that injurous accident coordinate data is underfiled with approximately the same distribution as non-injurous accident unfiling.

Said another way, there are so many records outstanding that we can simply assume that those records which *do* have coordinate data are a representative sample of records missing that data.

Any summary statistics that we gather based on geographical coordinality (borough, district, etc.) will be significant undercounts, but will nevertheless be a more than acceptable enough sample from which to draw conclusions about the underlying data.

At issue is the fact that the data on "Injuries" is plenty but that on "Deaths" is scarce.

With this in mind, I decided to procede by leaving "Injuries" on the side and populating coordinate data only for "Deaths".

In [19]:
def geolocate(srs):
    """
    Pandas ufunc which operates on a Pandas series, applying geolocation to tag coordinates from street name.
    Handles a variety of inputs:
    (PERSONS KILLED = 0) -> Does nothing.
    (PERSONS KILLED > 0, LOCATION defined, ON STREET NAME any, OFF STREET NAME any, CROSS STREET NAME any) -> Does nothing.
    (PERSONS KILLED > 0, LOCATION undef, ON STREET NAME def, OFF STREET NAME def, CROSS STREET NAME any) -> Geolocates using ON STREET and OFF STREET.
    (PERSONS KILLED > 0, LOCATION undef, ON STREET NAME undef, OFF STREET NAME undef, CROSS STREET NAME undef) -> Does nothing.*
    (PERSONS KILLED > 0, LOCATION undef, ON STREET NAME undef, OFF STREET NAME undef, CROSS STREET NAME def) -> Does nothing.**

    *  A cleanup step in the after stage might then remove these entries.
    ** This was found to be too unreliable to be worth trying.
    """
    on_street = srs['ON STREET NAME']
    cross_street = srs['CROSS STREET NAME']
    off_street = srs['OFF STREET NAME']
#     if str(srs['LOCATION']) == 'nan' and (srs['NUMBER OF PERSONS INJURED'] > 0 or srs['NUMBER OF PERSONS KILLED'] > 0):
    if str(srs['LOCATION']) == 'nan' and (srs['NUMBER OF PERSONS KILLED'] > 0):
        if str(on_street) != 'nan' and str(cross_street) != 'nan':
            g = geocoder.google('{0} and {1}, New York City'.format(on_street, cross_street))
            if g.status == 'ZERO_RESULTS':
                # Fail.
                pass
            # Success!
            srs['LOCATION'] = g.latlng
            srs['LATITUDE'] = g.latlng[0]
            srs['LONGITUDE'] = g.latlng[1]
#             print('{0} and {1}, New York City -> {2}'.format(on_street, cross_street, g.latlng))
        elif str(off_street) != 'nan':
            # off_street was found to be too inaccurate to be reliably returned.
            # Fail.
            pass
#             g = geocoder.google('{0}, New York City'.format(off_street))
#             if g.status == 'ZERO_RESULTS':
#                 pass
#             else:
#                 srs['LOCATION'] = g.latlng
        else:
            # Fail.
            pass
    return srs

In [20]:
filtered_fitted_df = filtered_df.copy()
filtered_fitted_df = filtered_fitted_df.apply(geolocate, axis=1)

In [21]:
filtered_fitted_df[filtered_fitted_df['NUMBER OF PERSONS KILLED'] > 0]

,DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
UNIQUE KEY,,,,,,,,,,,,,,,,,,,,,,,,,,,
3404362,03-11-2016 13:53,NaN,NaN,40.741691,-73.735405,"(40.7416909, -73.7354052)",NaN,NaN,NaN,3,1,0,0,0,0,3,1,Tire Failure/Inadequate,Unspecified,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
3403859,03-11-2016 08:17,QUEENS,11368.0,40.754707,-73.854786,"(40.7547066, -73.854786)",37 AVENUE,113 STREET,NaN,0,1,0,1,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
3402892,03-10-2016 19:27,BRONX,10452.0,40.835432,-73.923398,"(40.835432, -73.9233984)",JEROME AVENUE,SHAKESPEARE AVENUE,NaN,0,1,0,1,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,VAN,NaN,NaN,NaN,NaN
3400502,03-06-2016 11:29,NaN,NaN,40.843284,-73.837076,"[40.8432843, -73.8370762]",WESTCHESTER AVENUE,HUTCHINSON RIVER PARKWAY EAST,NaN,0,1,0,0,0,0,0,1,Unspecified,NaN,NaN,NaN,NaN,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
3399154,03-03-2016 02:28,STATEN ISLAND,10305.0,40.601204,-74.065076,"(40.6012042, -74.0650758)",NARROWS ROAD SOUTH,LILY POND AVENUE,NaN,0,1,0,0,0,0,0,1,Following Too Closely,NaN,NaN,NaN,NaN,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
3396423,02-29-2016 08:34,MANHATTAN,10016.0,40.749049,-73.982023,"(40.7490492, -73.982023)",EAST 36 STREET,MADISON AVENUE,NaN,0,1,0,1,0,0,0,0,Failure to Yield Right-of-Way,NaN,NaN,NaN,NaN,LIVERY VEHICLE,NaN,NaN,NaN,NaN
3395981,02-28-2016 01:25,NaN,NaN,40.844456,-73.902825,"[40.8444559, -73.9028254]",WEBSTER AVENUE,CROSS BRONX EXPRESSWAY,NaN,0,1,0,1,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
3396507,02-28-2016 23:28,BRONX,10462.0,40.835721,-73.855646,"(40.8357212, -73.8556463)",UNIONPORT ROAD,MCGRAW AVENUE,NaN,0,1,0,1,0,0,0,0,Passenger Distraction,NaN,NaN,NaN,NaN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
3396249,02-28-2016 04:40,QUEENS,11419.0,40.682461,-73.830015,"(40.6824607, -73.8300148)",111 STREET,107 AVENUE,NaN,0,1,0,1,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,UNKNOWN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN


At this point the data is as complete as it can be. Save it.

In [22]:
filtered_fitted_df.to_csv('NYPD Motor Vehicle Collisions Enriched.csv')